Import Libraries for Modelling





In [41]:
%%capture

!pip install creme scikit-learn pandas numpy joblib

Import Data

In [42]:
import pandas as pd

df = pd.read_csv('nakuru_water.csv')

df.head()

,Area,customers,customer_smart,day,time_of_day,weather,Price_of_water,lts_per_day
0,Golf Estate,London flat8,Golf park,tue,Morning,calm,46.928,2.60
1,Shabab,London flat1,London flat8,sat,lunch_hour,sunny/rainny,24.192,3.13
2,Golf Estate,London flat2,London flat4,tue,Morning,rainy,31.248,5.29
3,Olive,London flat1,Golf park,wen,midnight,sunny/windy,36.120,8.03
4,Shabab,London flat3,London flat4,wen,midnight,sunny,35.336,3.84


Split the Training and Testing set

In [43]:
import numpy as np
from sklearn.model_selection import train_test_split

In [44]:
x = df.loc[:,['day', 'Price_of_water', 'weather']]

x.head(2)

,day,Price_of_water,weather
0,tue,46.928,calm
1,sat,24.192,sunny/rainny


In [45]:
x.shape

(9999, 3)

In [46]:
y = df.loc[:,['Area', 'time_of_day']]

In [47]:
y.head(2)

,Area,time_of_day
0,Golf Estate,Morning
1,Shabab,lunch_hour


In [48]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)


In [49]:
y.shape

(9999, 2)

Fveature Engineering X and Y

In [50]:
from sklearn.pipeline import make_pipeline 
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder

In [51]:
xcols = [col for col in x]

xcols

['day', 'Price_of_water', 'weather']

In [52]:
from sklearn.preprocessing import StandardScaler

In [53]:
pq = make_pipeline(OneHotEncoder(sparse = False, handle_unknown = "ignore"))

In [54]:
Ct = make_column_transformer((pq, xcols), remainder=StandardScaler())

Model Selection

In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

model = RandomForestClassifier()

In [56]:
pipe = make_pipeline(Ct, model)

In [57]:
pipe1 = pipe.fit(x_train, y_train)

In [58]:
pipe1

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('pipeline',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['day', 'Price_of_water',
                                                   'weather'])])),
                ('randomforestclassifier', RandomForestClassifier())])

In [59]:
#get a sample from the dataset.
samp = x.sample(1, random_state = 130)

samp

,day,Price_of_water,weather
4430,fri,32.648,sunny/windy


In [60]:
pipe1.predict(samp)

array([['Shabab', 'evening']], dtype=object)

In [61]:
# from sklearn.metrics import accuracy_score

# y_pred = pipe1.predict(x_test)

# accuracy_score(y_test, y_pred)

In [62]:
#saving the model
import joblib
joblib.dump(pipe1, "modell")

['modell']

In [63]:
#load our model
# !pip install joblib
Model = joblib.load("modell")

In [64]:
#creating out of sample data
water = {'day' : 'sat', 'Price_of_water' : '66,75', 'weather' : 'sunny'}


In [65]:
#convert to pandas DT
test = pd.DataFrame(water, index = [289])

test

,day,Price_of_water,weather
289,sat,"66,75",sunny


In [66]:
#make prediction
Model.predict(test)

array([['Shabab', 'evening']], dtype=object)